In [13]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time
import pandas as pd

# Define the IB API app
class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []  # Initialize an empty list to store data

    # Override the historicalData function to process and store incoming data
    def historicalData(self, reqId, bar):
        # Append the data as a dictionary to self.data
        self.data.append({
            "Date": bar.date,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume
        })

    def historicalDataEnd(self, reqId, start, end):
        print("Historical Data Ended")
        # Convert the data to a DataFrame when data collection is complete
        self.df = pd.DataFrame(self.data)
        print(self.df)  # Display the DataFrame to verify
        self.disconnect()  # Disconnect after data collection is complete

# Define the app handler for running in the notebook
class IBApp:
    def __init__(self):
        self.app = IBApi()

    def connect(self):
        self.app.connect("127.0.0.1", 7496, 0)  # Change port if needed
        thread = threading.Thread(target=self.run_app, daemon=True)
        thread.start()
        time.sleep(1)  # Allow time for the connection to establish

    def run_app(self):
        self.app.run()

    def request_oil_data(self):
        # Define the contract for Crude Oil Futures
        contract = Contract()
        contract.symbol = "CL"
        contract.secType = "FUT"
        contract.exchange = "NYMEX"
        contract.currency = "USD"
        contract.lastTradeDateOrContractMonth = "202412"  # Example: Dec 2024 contract

        # Request historical data
        self.app.reqHistoricalData(
            reqId=1,
            contract=contract,
            endDateTime='',
            durationStr='1 D',  # 1 month
            barSizeSetting='5 mins',
            whatToShow='TRADES',
            useRTH=0,
            formatDate=1,
            keepUpToDate=False,
            chartOptions=[]
        )

    def disconnect(self):
        self.app.disconnect()

# Create an instance and connect
app = IBApp()
app.connect()

# Request data and output to a DataFrame
app.request_oil_data()

# Wait for data retrieval to complete
time.sleep(10)

# Access the DataFrame
df = app.app.df if hasattr(app.app, 'df') else pd.DataFrame()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


Historical Data Ended
                  Date   Open   High    Low  Close  Volume
0   20241030  18:00:00  69.10  69.10  68.96  69.02     378
1   20241030  18:05:00  69.02  69.07  69.01  69.05      99
2   20241030  18:10:00  69.06  69.07  69.01  69.01     103
3   20241030  18:15:00  69.01  69.02  69.00  69.00      54
4   20241030  18:20:00  69.01  69.01  68.99  69.00      25
5   20241030  18:25:00  69.00  69.05  69.00  69.04      40
6   20241030  18:30:00  69.05  69.05  69.03  69.03      63
7   20241030  18:35:00  69.03  69.03  69.00  69.00      64
8   20241030  18:40:00  68.99  69.01  68.98  68.99      60
9   20241030  18:45:00  68.99  68.99  68.95  68.97      66
10  20241030  18:50:00  68.97  69.00  68.96  68.99      44
11  20241030  18:55:00  68.98  68.98  68.97  68.98      23
12  20241030  19:00:00  68.98  69.02  68.98  69.01      48
13  20241030  19:05:00  69.02  69.03  69.00  69.01      31
14  20241030  19:10:00  69.02  69.02  69.00  69.00      22
15  20241030  19:15:00  69.00  69.

In [17]:
data = df.to_csv()